In [5]:
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
import joblib

import datetime,time
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(rc={'figure.figsize':(12,3)})
pd.set_option('display.max_columns', 50)
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split, KFold,cross_val_score,cross_validate,GridSearchCV
from lightgbm import LGBMRegressor,LGBMClassifier
#from catboost import CatBoostRegressor,CatBoostClassifier
from xgboost import XGBRegressor,XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder,PolynomialFeatures
le=LabelEncoder()
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import SplineTransformer
plt.style.use("seaborn-whitegrid")
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm_notebook
import os

In [6]:
leak_1_15m = pd.DataFrame(np.random.permutation(np.load("Data/Leak 1_15m.npy")), 
                          columns=["pressure_in", "flow_in", "pressure_out", "flow_out"]).iloc[:1000,:]
leak_1_30m = pd.DataFrame(np.random.permutation(np.load("Data/Leak 1_30m.npy")), 
                          columns=["pressure_in", "flow_in", "pressure_out", "flow_out"]).iloc[:1000,:]
leak_1_50m = pd.DataFrame(np.random.permutation(np.load("Data/Leak 1_50m.npy")), 
                          columns=["pressure_in", "flow_in", "pressure_out", "flow_out"]).iloc[:1000,:]

leak_2_15m = pd.DataFrame(np.random.permutation(np.load("Data/Leak 1_15m.npy")), 
                          columns=["pressure_in", "flow_in", "pressure_out", "flow_out"]).iloc[:1000,:]
leak_2_30m = pd.DataFrame(np.random.permutation(np.load("Data/Leak 1_30m.npy")), 
                          columns=["pressure_in", "flow_in", "pressure_out", "flow_out"]).iloc[:1000,:]
leak_2_50m = pd.DataFrame(np.random.permutation(np.load("Data/Leak 1_50m.npy")), 
                          columns=["pressure_in", "flow_in", "pressure_out", "flow_out"]).iloc[:1000,:]

leak_3_15m = pd.DataFrame(np.random.permutation(np.load("Data/Leak 1_15m.npy")), 
                          columns=["pressure_in", "flow_in", "pressure_out", "flow_out"]).iloc[:1000,:]
leak_3_30m = pd.DataFrame(np.random.permutation(np.load("Data/Leak 1_30m.npy")), 
                          columns=["pressure_in", "flow_in", "pressure_out", "flow_out"]).iloc[:1000,:]
leak_3_50m = pd.DataFrame(np.random.permutation(np.load("Data/Leak 1_50m.npy")), 
                          columns=["pressure_in", "flow_in", "pressure_out", "flow_out"]).iloc[:1000,:]


In [7]:
leak_1_15m['leak_point']=1
leak_1_15m["leak_distance_(metre)"]=15
leak_1_30m['leak_point']=1
leak_1_30m["leak_distance_(metre)"]=30
leak_1_50m['leak_point']=1
leak_1_50m["leak_distance_(metre)"]=50

leak_2_15m['leak_point']=2
leak_2_15m["leak_distance_(metre)"]=15
leak_2_30m['leak_point']=2
leak_2_30m["leak_distance_(metre)"]=30
leak_2_50m['leak_point']=2
leak_2_50m["leak_distance_(metre)"]=50

leak_3_15m['leak_point']=3
leak_3_15m["leak_distance_(metre)"]=15
leak_3_30m['leak_point']=3
leak_3_30m["leak_distance_(metre)"]=30
leak_3_50m['leak_point']=3
leak_3_50m["leak_distance_(metre)"]=50

In [8]:

df = pd.DataFrame(np.concatenate([leak_1_15m,leak_1_30m,leak_1_50m,\
                                  leak_2_15m, leak_2_30m, leak_2_50m,\
leak_3_15m, leak_3_30m, leak_3_50m], axis=0),
columns=["pressure_in", "flow_in", "pressure_out", "flow_out","leak_point","leak_distance_(metre)"])
df.to_csv('sample.csv',index=False)
df


,pressure_in,flow_in,pressure_out,flow_out,leak_point,leak_distance_(metre)
0,40.443912,0.213503,40.639994,0.102610,1.0,15.0
1,40.147398,0.223741,40.494760,0.111342,1.0,15.0
2,40.569851,0.217233,40.325586,0.105920,1.0,15.0
3,39.997547,0.236606,40.314414,0.126976,1.0,15.0
4,39.978417,0.203260,40.520295,0.089397,1.0,15.0
...,...,...,...,...,...,...
8995,39.348822,0.271964,38.824911,0.183020,3.0,50.0
8996,39.498323,0.232993,38.347457,0.143454,3.0,50.0
8997,39.781419,0.244353,38.595733,0.153158,3.0,50.0
8998,39.270891,0.263369,38.309261,0.171864,3.0,50.0


In [9]:
def summary(df):
    sum = pd.DataFrame(df.dtypes, columns=['dtypes'])
    sum['missing_num'] = df.isna().sum().values*100
    sum['missing%'] = (df.isna().sum().values*100)/len(df)
    sum['uniques'] = df.nunique().values
    sum['count'] = df.count().values
    desc = pd.DataFrame(df.describe().T)
    sum['min'] = desc['min']
    sum['max'] = desc['max']
    sum['mean'] = desc['mean']
    sum['75%'] = desc['75%']
    return sum

summary(df).style.background_gradient(cmap='Blues')

,dtypes,missing_num,missing%,uniques,count,min,max,mean,75%
pressure_in,float64,0,0.000000,2372,9000,37.222418,40.992304,39.373191,40.050588
flow_in,float64,0,0.000000,8879,9000,0.199136,0.815785,0.397225,0.687103
pressure_out,float64,0,0.000000,2215,9000,28.914572,41.281577,36.393843,40.319202
flow_out,float64,0,0.000000,8924,9000,0.085862,0.542662,0.237903,0.410817
leak_point,float64,0,0.000000,3,9000,1.000000,3.000000,2.000000,3.000000
leak_distance_(metre),float64,0,0.000000,3,9000,15.000000,50.000000,31.666667,50.000000


In [10]:

#df.fillna(df.mean(),inplace=True)

In [11]:
#checking for correlations,which col to be droprd or maintain

top20_corrs = abs(df.corr()['flow_out']).sort_values(ascending = False).head(30)
top20_corrs

flow_out                 1.000000
flow_in                  0.994186
pressure_out             0.985464
pressure_in              0.907312
leak_distance_(metre)    0.051677
leak_point               0.001259
Name: flow_out, dtype: float64

In [12]:
# Trying to remove redundant features
def redundant_feature(df):
    redundant = []
    for i in df.columns:
        counts = df[i].value_counts()
        count_max = counts.iloc[0]
        if count_max / len(df) * 100 > 99:
            redundant.append(i)
    redundant = list(redundant)
    return redundant
redundant_features = redundant_feature(df)
redundant_features

[]

In [13]:
df.replace([np.inf, -np.inf], 0, inplace=True)

In [14]:
df['leak_point'].value_counts()

1.0    3000
2.0    3000
3.0    3000
Name: leak_point, dtype: int64

# modelling, variation of validation and so on----

In [18]:
df

,pressure_in,flow_in,pressure_out,flow_out,leak_point,leak_distance_(metre)
0,40.443912,0.213503,40.639994,0.102610,1.0,15.0
1,40.147398,0.223741,40.494760,0.111342,1.0,15.0
2,40.569851,0.217233,40.325586,0.105920,1.0,15.0
3,39.997547,0.236606,40.314414,0.126976,1.0,15.0
4,39.978417,0.203260,40.520295,0.089397,1.0,15.0
...,...,...,...,...,...,...
8995,39.348822,0.271964,38.824911,0.183020,3.0,50.0
8996,39.498323,0.232993,38.347457,0.143454,3.0,50.0
8997,39.781419,0.244353,38.595733,0.153158,3.0,50.0
8998,39.270891,0.263369,38.309261,0.171864,3.0,50.0


In [19]:
col=df.drop([["leak_point","leak_distance_(metre)"]],axis=1).columns
from sklearn.model_selection  import train_test_split
X=df[col]
y = df["leak_point","leak_distance_(metre)"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

KeyError: "[('leak_point', 'leak_distance_(metre)')] not found in axis"

In [11]:
rf_pipe_model = make_pipeline(
    SimpleImputer(strategy="most_frequent").fit(X,y),
    StandardScaler().fit(X,y),
    RandomForestRegressor().fit(X,y)# CatBoostRegressor(**cn).fit(X,y)
)

In [12]:
y_predicted = rf_pipe_model.predict(X_test)

RMSE = np.sqrt(mean_squared_error(y_test, (y_predicted)))

print("Root Mean Square Error:\n")
print(RMSE)#4837628.851648336

Root Mean Square Error:

0.1404073374916445


In [13]:
# Analyse predictions
pred_errors = X_test.copy()
pred_errors['target'] = y_test
pred_errors['prediction'] = y_predicted
pred_errors['error'] = abs(pred_errors.prediction - pred_errors.target)
pred_errors = pred_errors[[ 'target', 'prediction', 'error']]
pred_errors.sort_values(by = 'error', ascending = False, inplace = True)
pred_errors.head()

,target,prediction,error
7598,0.341867,0.546495,0.204628
4816,0.343005,0.546495,0.203491
1710,0.343147,0.546495,0.203348
3754,0.085477,0.287140,0.201663
782,0.086518,0.287140,0.200622


In [14]:
pred_errors.describe()

,target,prediction,error
count,2970.000000,2970.000000,2970.000000
mean,0.235130,0.371583,0.136456
std,0.140951,0.121553,0.033080
min,0.085477,0.287140,0.004645
25%,0.124093,0.287140,0.114203
50%,0.162495,0.287140,0.133977
75%,0.408334,0.546495,0.165188
max,0.551140,0.546495,0.204628


In [15]:
import pickle
data = {"model": rf_pipe_model}
with open('saved_steps.pkl', 'wb') as file:
    pickle.dump(data, file)

In [16]:
with open('saved_steps.pkl', 'rb') as file:
    data = pickle.load(file)

In [19]:

import optuna
from catboost import CatBoostRegressor
X_train, X_valid, y_train, y_valid= train_test_split(X, y, test_size=0.2, random_state=123)
# Define the objective function for Optuna
def catboost_objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 100),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 10)
    }

    model = CatBoostRegressor(**params, silent=True)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=False)
    predictions = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, predictions, squared=False)

    return rmse

# Create an Optuna study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(catboost_objective, n_trials=100)

# Print the best hyperparameters and the corresponding RMSE
best_params = study.best_params
best_rmse = study.best_value
print(f"CatBoost Best Hyperparameters: {best_params}")
print(f"CatBoost Best RMSE: {best_rmse}")#513


[I 2024-02-07 14:33:02,084] A new study created in memory with name: no-name-3824c31b-4f64-45f4-b894-0bb7d525790b
[I 2024-02-07 14:33:06,392] Trial 0 finished with value: 5220006.771389417 and parameters: {'learning_rate': 0.12532821352612716, 'n_estimators': 164, 'max_depth': 12, 'l2_leaf_reg': 0.2725557586708557, 'subsample': 0.9788172912196347, 'colsample_bylevel': 0.7108467637748328, 'min_child_samples': 3}. Best is trial 0 with value: 5220006.771389417.
[I 2024-02-07 14:33:07,811] Trial 1 finished with value: 5318626.885151328 and parameters: {'learning_rate': 0.018368066177399472, 'n_estimators': 129, 'max_depth': 8, 'l2_leaf_reg': 3.607059097177849, 'subsample': 0.9709262732056526, 'colsample_bylevel': 0.6367165202399643, 'min_child_samples': 2}. Best is trial 0 with value: 5220006.771389417.
[I 2024-02-07 14:33:09,582] Trial 2 finished with value: 5212367.687113526 and parameters: {'learning_rate': 0.06755691316701246, 'n_estimators': 275, 'max_depth': 10, 'l2_leaf_reg': 2.0261

[I 2024-02-07 14:36:21,074] Trial 24 finished with value: 5182669.379751944 and parameters: {'learning_rate': 0.27408968978781606, 'n_estimators': 393, 'max_depth': 5, 'l2_leaf_reg': 0.0019217370143835612, 'subsample': 0.6139555992273029, 'colsample_bylevel': 0.5046134285602778, 'min_child_samples': 5}. Best is trial 16 with value: 5168058.88652471.
[I 2024-02-07 14:36:21,950] Trial 25 finished with value: 5167587.486261883 and parameters: {'learning_rate': 0.08978572755473152, 'n_estimators': 293, 'max_depth': 6, 'l2_leaf_reg': 0.14362998196802826, 'subsample': 0.6736877184807665, 'colsample_bylevel': 0.5724367937443147, 'min_child_samples': 3}. Best is trial 25 with value: 5167587.486261883.
[I 2024-02-07 14:36:23,158] Trial 26 finished with value: 5196842.414002198 and parameters: {'learning_rate': 0.05217367330041077, 'n_estimators': 370, 'max_depth': 6, 'l2_leaf_reg': 0.1269330606462965, 'subsample': 0.6911117460138573, 'colsample_bylevel': 0.5840071088024519, 'min_child_samples':

[I 2024-02-07 14:36:55,656] Trial 48 finished with value: 5177062.893139654 and parameters: {'learning_rate': 0.029934711229354757, 'n_estimators': 191, 'max_depth': 9, 'l2_leaf_reg': 0.00926126152946739, 'subsample': 0.5301541562483196, 'colsample_bylevel': 0.5310318739520694, 'min_child_samples': 5}. Best is trial 30 with value: 5137779.83313612.
[I 2024-02-07 14:37:02,818] Trial 49 finished with value: 5214303.031814911 and parameters: {'learning_rate': 0.04081290854124414, 'n_estimators': 380, 'max_depth': 11, 'l2_leaf_reg': 2.9710506618776165e-05, 'subsample': 0.6575463542082883, 'colsample_bylevel': 0.6247390257591419, 'min_child_samples': 9}. Best is trial 30 with value: 5137779.83313612.
[I 2024-02-07 14:37:03,648] Trial 50 finished with value: 5206258.721177736 and parameters: {'learning_rate': 0.07362496475804924, 'n_estimators': 483, 'max_depth': 7, 'l2_leaf_reg': 0.001035488411987959, 'subsample': 0.634783836263889, 'colsample_bylevel': 0.747222787461367, 'min_child_samples

[I 2024-02-07 14:37:25,318] Trial 72 finished with value: 5238620.686560578 and parameters: {'learning_rate': 0.1215470449900351, 'n_estimators': 292, 'max_depth': 4, 'l2_leaf_reg': 99.15176422828185, 'subsample': 0.5257783964073711, 'colsample_bylevel': 0.5724710447403166, 'min_child_samples': 5}. Best is trial 30 with value: 5137779.83313612.
[I 2024-02-07 14:37:42,568] Trial 73 finished with value: 5389016.357149561 and parameters: {'learning_rate': 0.09780983711095526, 'n_estimators': 271, 'max_depth': 14, 'l2_leaf_reg': 0.0006999647598567784, 'subsample': 0.7307731645346518, 'colsample_bylevel': 0.5193520908578068, 'min_child_samples': 6}. Best is trial 30 with value: 5137779.83313612.
[I 2024-02-07 14:37:43,389] Trial 74 finished with value: 5221571.141150096 and parameters: {'learning_rate': 0.08012124343533443, 'n_estimators': 312, 'max_depth': 3, 'l2_leaf_reg': 13.747944232185636, 'subsample': 0.6732338193587003, 'colsample_bylevel': 0.5970026206564834, 'min_child_samples': 4}

[I 2024-02-07 14:38:00,652] Trial 96 finished with value: 5163873.472341665 and parameters: {'learning_rate': 0.07851259529604807, 'n_estimators': 268, 'max_depth': 8, 'l2_leaf_reg': 0.025618340484347902, 'subsample': 0.5499288430623255, 'colsample_bylevel': 0.547633027427235, 'min_child_samples': 5}. Best is trial 30 with value: 5137779.83313612.
[I 2024-02-07 14:38:01,554] Trial 97 finished with value: 5194091.619912167 and parameters: {'learning_rate': 0.06224946522356557, 'n_estimators': 270, 'max_depth': 8, 'l2_leaf_reg': 0.00046239113473102705, 'subsample': 0.5536072699013015, 'colsample_bylevel': 0.5508273770147064, 'min_child_samples': 8}. Best is trial 30 with value: 5137779.83313612.
[I 2024-02-07 14:38:03,356] Trial 98 finished with value: 5215816.281720471 and parameters: {'learning_rate': 0.0835788344674607, 'n_estimators': 204, 'max_depth': 9, 'l2_leaf_reg': 3.0461945344792785, 'subsample': 0.5356909019104041, 'colsample_bylevel': 0.5407553878483436, 'min_child_samples': 

CatBoost Best Hyperparameters: {'learning_rate': 0.11889668184372702, 'n_estimators': 464, 'max_depth': 8, 'l2_leaf_reg': 0.00014827946407480543, 'subsample': 0.6327183879497226, 'colsample_bylevel': 0.5574882165742737, 'min_child_samples': 7}
CatBoost Best RMSE: 5137779.83313612


In [35]:
import optuna
import xgboost as xgb

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def xgb_objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)
    }

    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=False)
    predictions = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, predictions, squared=False)

    return rmse

# Create an Optuna study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(xgb_objective, n_trials=100)

# Print the best hyperparameters and the corresponding RMSE
best_params = study.best_params
best_rmse = study.best_value
print(f"XGBoost Best Hyperparameters: {best_params}")
print(f"XGBoost Best RMSE: {best_rmse}")#489


[I 2024-02-07 14:42:35,118] A new study created in memory with name: no-name-ce9e1800-ca84-40c9-99b5-d92a00fbde1e
[I 2024-02-07 14:42:35,699] Trial 0 finished with value: 5112032.628975296 and parameters: {'learning_rate': 0.04742097017171625, 'n_estimators': 319, 'max_depth': 11, 'min_child_weight': 3, 'subsample': 0.7863179895676273, 'colsample_bytree': 0.7270904904521516}. Best is trial 0 with value: 5112032.628975296.
[I 2024-02-07 14:42:36,176] Trial 1 finished with value: 4962651.848601633 and parameters: {'learning_rate': 0.03424927451003665, 'n_estimators': 175, 'max_depth': 3, 'min_child_weight': 3, 'subsample': 0.6033856159358004, 'colsample_bytree': 0.7822201256094187}. Best is trial 1 with value: 4962651.848601633.
[I 2024-02-07 14:42:37,972] Trial 2 finished with value: 5007722.030442618 and parameters: {'learning_rate': 0.010079531805218457, 'n_estimators': 279, 'max_depth': 12, 'min_child_weight': 5, 'subsample': 0.6449927018115651, 'colsample_bytree': 0.7569146493731396

[I 2024-02-07 14:42:49,158] Trial 26 finished with value: 4939244.703823023 and parameters: {'learning_rate': 0.027656250499734913, 'n_estimators': 268, 'max_depth': 6, 'min_child_weight': 6, 'subsample': 0.568924614749636, 'colsample_bytree': 0.674849682651769}. Best is trial 13 with value: 4918620.653935615.
[I 2024-02-07 14:42:50,144] Trial 27 finished with value: 4973512.774049892 and parameters: {'learning_rate': 0.01367732755802917, 'n_estimators': 253, 'max_depth': 9, 'min_child_weight': 9, 'subsample': 0.6382535178953714, 'colsample_bytree': 0.6592022994890948}. Best is trial 13 with value: 4918620.653935615.
[I 2024-02-07 14:42:51,086] Trial 28 finished with value: 4952131.165293197 and parameters: {'learning_rate': 0.01597893090544001, 'n_estimators': 362, 'max_depth': 4, 'min_child_weight': 7, 'subsample': 0.9072930555852392, 'colsample_bytree': 0.5956066617567373}. Best is trial 13 with value: 4918620.653935615.
[I 2024-02-07 14:42:51,447] Trial 29 finished with value: 5000

[I 2024-02-07 14:43:07,969] Trial 53 finished with value: 4903751.622238461 and parameters: {'learning_rate': 0.020857510079799628, 'n_estimators': 344, 'max_depth': 5, 'min_child_weight': 9, 'subsample': 0.6298525757265532, 'colsample_bytree': 0.7658009013242438}. Best is trial 51 with value: 4891722.84664151.
[I 2024-02-07 14:43:08,677] Trial 54 finished with value: 4916536.2971847635 and parameters: {'learning_rate': 0.019830740211185503, 'n_estimators': 355, 'max_depth': 6, 'min_child_weight': 8, 'subsample': 0.5230977574671851, 'colsample_bytree': 0.7512037556481775}. Best is trial 51 with value: 4891722.84664151.
[I 2024-02-07 14:43:09,270] Trial 55 finished with value: 4935056.077350883 and parameters: {'learning_rate': 0.02899107021648863, 'n_estimators': 340, 'max_depth': 5, 'min_child_weight': 9, 'subsample': 0.7500260086986762, 'colsample_bytree': 0.7020000702052267}. Best is trial 51 with value: 4891722.84664151.
[I 2024-02-07 14:43:10,005] Trial 56 finished with value: 494

[I 2024-02-07 14:43:27,238] Trial 80 finished with value: 4919128.626535507 and parameters: {'learning_rate': 0.03371869436845477, 'n_estimators': 366, 'max_depth': 4, 'min_child_weight': 10, 'subsample': 0.6896409242912466, 'colsample_bytree': 0.8135679886741257}. Best is trial 51 with value: 4891722.84664151.
[I 2024-02-07 14:43:27,923] Trial 81 finished with value: 4912095.005584606 and parameters: {'learning_rate': 0.02068256213736588, 'n_estimators': 335, 'max_depth': 5, 'min_child_weight': 9, 'subsample': 0.577909689969506, 'colsample_bytree': 0.7363583973382748}. Best is trial 51 with value: 4891722.84664151.
[I 2024-02-07 14:43:28,890] Trial 82 finished with value: 4914449.406897554 and parameters: {'learning_rate': 0.021159331479071508, 'n_estimators': 331, 'max_depth': 5, 'min_child_weight': 9, 'subsample': 0.6048247437271858, 'colsample_bytree': 0.7598073281516331}. Best is trial 51 with value: 4891722.84664151.
[I 2024-02-07 14:43:29,813] Trial 83 finished with value: 49333

XGBoost Best Hyperparameters: {'learning_rate': 0.023062124011033285, 'n_estimators': 342, 'max_depth': 5, 'min_child_weight': 9, 'subsample': 0.6023006461447127, 'colsample_bytree': 0.8040270013877765}
XGBoost Best RMSE: 4891722.84664151


In [36]:

import lightgbm as lgb
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def lgb_objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'num_leaves': trial.suggest_int('num_leaves', 10, 200),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 100),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 100)
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train,y_train,eval_metric='rmse')
    predictions = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, predictions, squared=False)

    return rmse

# Create an Optuna study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(lgb_objective, n_trials=100)

# Print the best hyperparameters and the corresponding RMSE
best_params = study.best_params
best_rmse = study.best_value
print(f"LightGBM Best Hyperparameters: {best_params}")
print(f"LightGBM Best RMSE: {best_rmse}")#491

[I 2024-02-07 14:43:41,471] A new study created in memory with name: no-name-a1916523-ecb8-4f44-a6a0-42b6c9438479


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000549 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 230
[LightGBM] [Info] Number of data points in the train set: 3847, number of used features: 21
[LightGBM] [Info] Start training from score 6732170.600374
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

[I 2024-02-07 14:43:42,494] Trial 0 finished with value: 5318469.722829551 and parameters: {'learning_rate': 0.07811753107977776, 'n_estimators': 223, 'max_depth': 11, 'num_leaves': 135, 'min_child_samples': 6, 'subsample': 0.5343788496861525, 'colsample_bytree': 0.6175834942530394, 'reg_lambda': 3.639064864920674e-05, 'reg_alpha': 5.700560373572772e-05}. Best is trial 0 with value: 5318469.722829551.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000664 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 230
[LightGBM] [Info] Number of data points in the train set: 3847, number of used features: 21
[LightGBM] [Info] Start training from score 6732170.600374
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000591 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 230
[LightGBM] [Info] Number of data points in the train set: 3847, number of used features: 21
[LightGBM] [Info] Start training from score 6732170.600374
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000672 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 230
[LightGBM] [Info] Number of data points in the train set: 3847, number 

[I 2024-02-07 14:44:16,765] Trial 98 finished with value: 4973207.661635976 and parameters: {'learning_rate': 0.024181813615065155, 'n_estimators': 293, 'max_depth': 5, 'num_leaves': 31, 'min_child_samples': 15, 'subsample': 0.8784234510105533, 'colsample_bytree': 0.69986488031162, 'reg_lambda': 0.0001960840778388409, 'reg_alpha': 7.434007993295266e-05}. Best is trial 91 with value: 4916515.888259107.


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000425 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 230
[LightGBM] [Info] Number of data points in the train set: 3847, number of used features: 21
[LightGBM] [Info] Start training from score 6732170.600374
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

[I 2024-02-07 14:44:17,045] Trial 99 finished with value: 4948919.200136811 and parameters: {'learning_rate': 0.01921441755406671, 'n_estimators': 256, 'max_depth': 4, 'num_leaves': 23, 'min_child_samples': 20, 'subsample': 0.9424784673491913, 'colsample_bytree': 0.7293108849483104, 'reg_lambda': 0.00010253627133214834, 'reg_alpha': 3.260988167620832e-05}. Best is trial 91 with value: 4916515.888259107.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [54]:
kf = KFold(n_splits=5,shuffle=False)
model = CatBoostRegressor(**c)
RMSE = []
R2=[]
c_train_pred = np.zeros(len(X))
c_test_pred = np.zeros(len(test))
for (train_index,test_index) in kf.split(X,y):
    X_train,X_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = y.iloc[train_index],y.iloc[test_index]

    model.fit(X_train,y_train,early_stopping_rounds=500,eval_set=[(X_test,y_test)],verbose=False)
    RMSE.append(np.sqrt(mean_squared_error(model.predict(X_test),y_test)))
    R2.append(r2_score(model.predict(X_test),y_test))

    c_train_pred+=model.predict(X)
    c_test_pred+=model.predict(test[col])
    c_predictions= np.absolute(c_test_pred/kf.n_splits)
    sub['power-generated']=c_predictions
    sub.to_csv('ctb_cv5.csv',index=False)


print(
f"r2 or accuracy:     {np.mean(R2)}\n"       #
f"Root Mean Squared Error: {np.mean(RMSE)} ")# 2896

r2 or accuracy:     0.0892151661305806
Root Mean Squared Error: 5138191.748552036 


In [55]:
import xgboost as xgb
model = xgb.XGBRegressor(**x)
kf = KFold(n_splits=5, shuffle=False)
RMSE = []
R2 = []
x_train_pred = np.zeros(len(X))
x_test_pred = np.zeros(len(test))

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train, early_stopping_rounds=500, eval_set=[(X_test, y_test)], verbose=False)
    predictions = model.predict(X_test)
    RMSE.append(np.sqrt(mean_squared_error(predictions, y_test)))
    R2.append(r2_score(predictions, y_test))
    x_train_pred[train_index] = model.predict(X_train)
    x_test_pred += model.predict(test[X_train.columns])
x_predictions = np.absolute(x_test_pred / kf.get_n_splits())
sub['power-generated'] = x_predictions
sub.to_csv('xgb_cv5.csv', index=False)
print(f"r2 or accuracy: {np.mean(R2)}")
print(f"Root Mean Squared Error: {np.mean(RMSE)}")


r2 or accuracy: 0.07729713877632916
Root Mean Squared Error: 5190631.537743181


In [56]:
kf = KFold(n_splits=5,shuffle=False)
model = LGBMRegressor(**l)
RMSE = []
R2=[]
l_train_pred = np.zeros(len(X))
l_test_pred = np.zeros(len(test))
for (train_index,test_index) in kf.split(X,y):
    X_train,X_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = y.iloc[train_index],y.iloc[test_index]

    model.fit(X_train,y_train,eval_metric='rmse')
    RMSE.append(np.sqrt(mean_squared_error(model.predict(X_test),y_test)))
    R2.append(r2_score(model.predict(X_test),y_test))

    l_train_pred+=model.predict(X)
    l_test_pred+=model.predict(test[col])
    l_predictions= np.absolute(l_test_pred/kf.n_splits)
    sub['power-generated']=l_predictions
    sub.to_csv('lgr_cv5.csv',index=False)
print(
f"r2 or accuracy:     {np.mean(R2)}\n"       #591322.5448032819
f"Root Mean Squared Error: {np.mean(RMSE)} ")#

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000665 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 228
[LightGBM] [Info] Number of data points in the train set: 3847, number of used features: 21
[LightGBM] [Info] Start training from score 6471818.617555
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000405 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 3847, number of used features: 21
[LightGBM] [Info] Start training from score 6608439.740053
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000445 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 237
[LightGBM] [Info] Number of data points in the train set: 3847, number of used features: 21
[LightGBM] [Info] Start training from score 6659054.833295
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000431 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 232
[LightGBM] [Info] Number of data points in the train set: 3847, number of used features: 21
[LightGBM] [Info] Start training from score 6563808.416835
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000387 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 228
[LightGBM] [Info] Number of data points in the train set: 3848, number of used features: 21
[LightGBM] [Info] Start training from score 7040871.375382
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

r2 or accuracy:     0.07678336150496498
Root Mean Squared Error: 5151662.193899335 
